In [ ]:
!pip install muspy
!sudo apt install -y fluidsynth
!pip install --upgrade pyfluidsynth

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.1/119.1 KB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.8/22.8 MB 67.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 KB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 103.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 KB 4.1 MB/s eta 0:00:00
  Created wheel for pretty-midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592303 sha256=7730c56f3e50c0957a12be08fa050b753e1487e257b6d15b4d9c5ab7e834467e
  Stored in directory: /root/.cache/pip/wheels/75/ec/20/b8e937a5bcf1de547ea5ce465db7de7f6761e15e6f0a01e25f
Successfully built pretty-midi
  Attempting uninstall: music21
    Found existing installation: music21 5.5.0
    Uninstalling music21-5.5.0:
      Successfully uninstalled music21-5.5.0
R

In [ ]:
import muspy as mp
import music21
import os

import random
import numpy
import torch
import torch.nn as nn
import torch.nn.functional as F
from pathlib import Path
import shutil
import glob
from progressbar import ProgressBar


import collections
import datetime
import fluidsynth
import numpy as np
import pathlib
import pandas as pd
import pretty_midi
import seaborn as sns
import tensorflow as tf

from IPython import display
from matplotlib import pyplot as plt
from typing import Dict, List, Optional, Sequence, Tuple

## Funcs

Taille de la suite de notes à considérer

In [ ]:
SIZE_OF_SEQ = 3

Fonctions utiles

In [ ]:
def create_empty_notes_list(n : int):
  dict_note = []
  for i in range(0,n):
    dict_note.append(0.0)
  return dict_note

def normalize_list(l : list):
  total = sum(l)
  if total > 0.0:
    for i in range(len(l)):
      l[i] = l[i]/total

def normalize_pitch_seq(notes : list):
  index_seq = 0
  power = len(notes) - 1
  for i in range(len(notes)):
    index_seq += (notes[i].pitch%12) * pow(12,power)
    power -= 1
  return index_seq

def decode_seq(index : int):
  power = SIZE_OF_SEQ - 1
  seq = []
  for i in range(3):
    a = index // pow(12,power)
    index = index % pow(12,power)
    seq.append(a)
    power -= 1
  return seq

## Génération matrice A et B

In [ ]:
list_notes = []
list_notes_dur = []

for i in range(0,pow(12,SIZE_OF_SEQ)): #on crée une liste qui, pour chaque suite de 3 notes, mettra la probabilité d'une note
  prob_allowed_notes = create_empty_notes_list(pow(12,SIZE_OF_SEQ))
  list_notes.append(prob_allowed_notes)

for i in range(850):
  prob_duration_notes = create_empty_notes_list(850)
  list_notes_dur.append(prob_duration_notes)

In [ ]:
data_dir = pathlib.Path("major_parts")
filenames = glob.glob(str(data_dir/'*.mid*'))
print('Number of files:', len(filenames))

Number of files: 326


In [ ]:
sample_file = filenames[20]
pm = pretty_midi.PrettyMIDI(sample_file)

In [ ]:
# Cellule qui permet de trouver la durée de la note la plus longue du dataset
# all_durs = []

# pbar_dur = ProgressBar()

# for filename in pbar_dur(os.listdir(data_dir)):
#     f = os.path.join(data_dir, filename)
#     # checking if it is a file
#     if os.path.isfile(f):
#       music = mp.from_music21(music21.converter.parse(f))
#       unsorted_notes = []
#       #assemblage de toutes les pistes de notes
#       for track in music.tracks:
#         m_notes = track.notes
#         if m_notes != []:
#           unsorted_notes += m_notes 
#       notes = sorted(unsorted_notes, key=lambda note: note.time)
#       if notes != []:
#         durations = [note.duration for note in notes]
#         all_durs.append(max(durations))
# print(max(all_durs))

In [ ]:
pbar = ProgressBar()

for filename in pbar(os.listdir(data_dir)):
    f = os.path.join(data_dir, filename)
    # checking if it is a file
    if os.path.isfile(f):
      music = mp.from_music21(music21.converter.parse(f))
      unsorted_notes = []
      #assemblage de toutes les pistes de notes
      for track in music.tracks:
        m_notes = track.notes
        if m_notes != []:
          unsorted_notes += m_notes 
      notes = sorted(unsorted_notes, key=lambda note: note.time)

      if notes != []:
      
        new_notes = [notes[0]]
        prev_notes = [notes[0]]
        prev_dur = notes[0].duration

        for i in range(1, len(notes)-1):
          curr_note = notes[i]
          curr_dur = notes[i].duration

          #si les deux notes sont éloignées, on ne les traite pas ensemble
          if (prev_notes[len(prev_notes)-1].time + prev_notes[len(prev_notes)-1].duration + 50) < curr_note.time:
            new_notes = [notes[i]]
            prev_notes = [notes[i]]
            prev_dur = notes[i].duration

          #si les deux notes sont rapprochées, on les traite ensemble
          else:
            new_notes.append(curr_note)
            if len(new_notes)==(SIZE_OF_SEQ+1):
              del new_notes[0]

            index_prev_pitches = normalize_pitch_seq(prev_notes)
            index_new_pitches = normalize_pitch_seq(new_notes)

            list_to_update = list_notes[index_new_pitches]
            list_to_update[index_prev_pitches] += 1

            list_to_update_dur = list_notes_dur[curr_dur]
            list_to_update_dur[prev_dur] += 1

            prev_notes = new_notes.copy()


      #list_to_update_dur = list_notes_dur[prev_dur] 
      #list_to_update_dur[notes[len(notes)-1].duration] += 1

100% (326 of 326) |######################| Elapsed Time: 0:23:27 Time:  0:23:27


## Génération d'une séquence de notes par HMM


In [ ]:
A = torch.tensor(list_notes)
A

tensor([[1930.,    0.,    0.,  ...,    0.,    0.,    0.],
        [  30.,    0.,    0.,  ...,    0.,    0.,    0.],
        [ 193.,    0.,    0.,  ...,    0.,    0.,    0.],
        ...,
        [   0.,    0.,    0.,  ...,    0.,    0.,   22.],
        [   0.,    0.,    0.,  ...,    0.,    0.,    3.],
        [   0.,    0.,    0.,  ...,    0.,    0.,   80.]])

In [ ]:
B =torch.tensor(numpy.transpose(list_notes_dur))
B

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64)

In [ ]:
L, V = torch.linalg.eig(A)
pi0 = V[:,0]
pi0 = pi0/torch.sum(pi0)
pi0 = torch.real(pi0)

In [ ]:
N=100 # taille de la séquence que l'on veut
# initialisation
current_state = torch.multinomial(A.diagonal(),1).item()

notes_index = [] #séquence des notes (pitch)
notes_index.append(current_state)

for i in range(N):
  # transition
  current_state = torch.multinomial(A[:,current_state],1).item()
  notes_index.append(current_state)

# optionnel
print(notes_index)

[628, 628, 630, 656, 968, 1254, 1224, 866, 33, 396, 1298, 35, 426, 1667, 1002, 1658, 890, 321, 401, 1358, 746, 318, 370, 986, 1469, 357, 835, 1380, 1013, 60, 729, 117, 1413, 1413, 1408, 1350, 650, 895, 375, 1055, 564, 1592, 99, 1195, 523, 1099, 1099, 1097, 1075, 811, 1103, 1140, 1584, 0, 4, 50, 602, 320, 386, 1180, 344, 683, 1288, 1640, 683, 1295, 1727, 1718, 1613, 351, 758, 456, 290, 27, 329, 500, 826, 1277, 1508, 823, 1243, 1099, 1097, 1073, 787, 808, 1062, 657, 976, 1349, 643, 811, 1097, 1073, 781, 740, 246, 1225, 877, 164, 244]


In [ ]:
Ld, Vd = torch.linalg.eig(B)
pi1 = Vd[:,0]
pi1 = pi1/torch.sum(pi1)
pi1 = torch.real(pi1)

In [ ]:
durs = [] #séquence de durées des notes
# initialisation
current_state = torch.multinomial(B.diagonal(),1).item()

durs.append(current_state)

for i in range(N):
  # transition
  current_state = torch.multinomial(B[:,current_state],1).item()
  durs.append(current_state)

# optionnel
print(durs)

[48, 6, 6, 6, 8, 24, 36, 6, 42, 24, 24, 30, 66, 96, 342, 12, 12, 162, 54, 12, 8, 6, 6, 8, 6, 6, 6, 24, 6, 6, 6, 12, 12, 80, 18, 8, 16, 48, 8, 8, 12, 6, 32, 48, 6, 8, 8, 8, 24, 32, 30, 12, 12, 12, 24, 12, 48, 32, 72, 36, 8, 6, 12, 18, 36, 6, 6, 8, 6, 8, 6, 6, 8, 12, 12, 32, 96, 8, 8, 6, 6, 8, 6, 8, 8, 6, 12, 30, 6, 8, 12, 12, 6, 96, 342, 12, 30, 16, 18, 12, 96]


## Génération de la partition

In [ ]:
generated_notes = []
prev_start = 0
prev_dur = 0
for i in range(len(notes_index)):
  pitch_seq = decode_seq(notes_index[i])
  pitch = pitch_seq[SIZE_OF_SEQ-1] + 72
  duration = 0.03 * durs[i]
  step = duration
  start = prev_start + step
  end = start + duration
  input_note = (pitch, step, duration)
  generated_notes.append((*input_note, start, end))
  prev_start = start

generated_notes = pd.DataFrame(
    generated_notes, columns=('pitch','step','duration', 'start', 'end'))

In [ ]:
instrument = pm.instruments[0]
instrument_name = pretty_midi.program_to_instrument_name(instrument.program)

In [ ]:
def display_audio(pm: pretty_midi.PrettyMIDI, seconds=45):
  waveform = pm.fluidsynth(fs=_SAMPLING_RATE)
  # Take a sample of the generated waveform to mitigate kernel resets
  waveform_short = waveform[:seconds*_SAMPLING_RATE]
  return display.Audio(waveform_short, rate=_SAMPLING_RATE)

def notes_to_midi(
  notes: pd.DataFrame,
  out_file: str, 
  instrument_name: str,
  velocity: int = 100,  # note loudness
) -> pretty_midi.PrettyMIDI:

  pm = pretty_midi.PrettyMIDI()
  instrument = pretty_midi.Instrument(
      program=pretty_midi.instrument_name_to_program(
          instrument_name))

  prev_start = 0
  for i, note in notes.iterrows():
    start = float(prev_start + note['step'])
    end = float(start + note['duration'])
    note = pretty_midi.Note(
        velocity=velocity,
        pitch=int(note['pitch']),
        start=start,
        end=end,
    )
    instrument.notes.append(note)
    prev_start = start

  pm.instruments.append(instrument)
  pm.write(out_file)
  return pm

In [ ]:
out_file = 'output.mid'
# Sampling rate for audio playback
_SAMPLING_RATE = 16000

out_pm = notes_to_midi(
    generated_notes, out_file=out_file, instrument_name=instrument_name)
display_audio(out_pm)